# DDSIM Primitives

Qiskit introduced the `qiskit.primitives` module in version 0.20.0 (Qiskit Terra 0.35.0) to provide a simplified way for end users to compute outputs of interest from a QuantumCircuit and to use backends.

The two main submodules of the primitives are the `Sampler` and the `Estimator`. The first one allows you to compute quasi-probability distributions from circuit measurements, while the second one defines an abstract interface for estimating the expectation value of an observable.

MQT now provides its own version of the Qiskit primitives:

- `DDSIMBackendSampler` leverages the already available `QasmSimulatorBackend` from DDSIM, while preserving the methods and functionality of Qiskit's sampler.

- `DDSIMEstimator` is currently in development and will be available soon.


## DDSIMBackendSampler

The `DDSIMBackendSampler` takes a list of `QuantumCircuit` objects and simulates them using the `QasmSimulatorBackend` from MQT DDSIM. It then computes the quasi-probability distributions of the circuits in the list and encapsulates the results, along with the job's metadata, within a `SamplerResult` object. 

Furthermore, it also handles transpilation and parameter binding when working with parametrized circuits.

Here we show an example on how to use this submodule:

In [ ]:
import numpy as np
from qiskit import *

from mqt.ddsim.sampler import DDSIMBackendSampler

# Circuit to create a Bell state
circ = QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(0, 2)
circ.measure_all()

# Show circuit
print(circ.draw(fold=-1))

# Initialize sampler
sampler = DDSIMBackendSampler()

# Submit job
job = sampler.run(circ)
result = job.result()

The `result()` method of the job returns a `SamplerResult` object, which includes both the quasi-probability distribution and job metadata.

In [ ]:
print(f">>> {result}")
print(f"  > Quasi-probability distribution: {result.quasi_dists[0]}")

You can also have a list with multiple quantum circuits as an input:

In [ ]:
# Create second circuit
circ2 = QuantumCircuit(3)
for qubit in circ2.qubits:
    circ2.h(qubit)
circ2.measure_all()

circuits = [circ, circ2]

[cir.draw(fold=-1) for cir in circuits]
job = sampler.run(circuits)
result = job.result()

print(f">>> Quasi-probability distribution: {result.quasi_dists}")

Or parametrized circuits:

In [ ]:
theta_a = Parameter("theta_a")
theta_b = Parameter("theta_b")
param_circ = QuantumCircuit(1)
param_circ.rx(theta_a, 0)
param_circ.rz(theta_b, 0)
param_circ.rx(theta_a, 0)
param_circ.barrier()
param_circ.measure_all()

job = sampler.run([circuit], [[np.pi / 2, np.pi]])
result = job.result()

print(f">>> Quasi-probability distribution: {result.quasi_dists}")